In [1]:
!pip install openai
!pip install pandas

In [23]:
from openai import OpenAI
import json
import random

In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd  /content/drive/MyDrive/ECE1786/Project

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/ECE1786/Project


In [4]:
 # API+Key
api_key = "API_KEY"

## Data Processing

### Recipe Data Initial Processing

In [17]:
def get_API_response(client,in_prompt,user_input,temp,topp):

  #print("temperature= {0}, top_p= {1} , max_tokens={2}, input_statement={3}".format(temp,topp,max_token,input_statement))

  chat_completion = client.chat.completions.create(
    messages=[
        {"role": "system", "content": in_prompt},
        {"role": "user", "content": user_input}
    ],
    model="gpt-4o",
    temperature = temp,
    top_p = topp
  )
  response = chat_completion.choices[0].message.content
  #print(chat_completion)

  return response

In [132]:
def process_API_res_get_processed_recipe(API_resonse,recipe_id,eachRecipe):
  try:
    processed_res= json.loads(API_resonse)
  except Exception as e:
      print(f"Error: {e}")
      print(API_resonse)

  processed_recipe = {
        "recipe_id": recipe_id,
        "recipe_title": eachRecipe["title"],
        "original_instructions": eachRecipe["instructions"],
        "ingredients": eachRecipe["ingredients"],
        "processed_output": processed_res
    }

  return processed_recipe


In [133]:
def get_processed_recipe_dataset(client,temp,topp,raw_recipe_dataset):

  out_list = []

  for recipe_id, eachRecipe in raw_recipe_dataset.items():

    #pass recipe to GPT
    ingredients_str = ', '.join(eachRecipe['ingredients'])
    prompt_recipe_process = recipe_process_prompt.format(eachRecipe["title"],eachRecipe["instructions"],ingredients_str)
    response = get_API_response(client, in_prompt=prompt_recipe_process, user_input="", temp=temp, topp=topp)
    #print(response)
    # get processed recipe
    processed_recipe =  process_API_res_get_processed_recipe(response,recipe_id,eachRecipe)
    out_list.append(processed_recipe)

  return out_list

In [134]:
#Get testing file
with open("sampled_test_data.json", "r") as f:
    recipe_dataset = json.load(f)

In [141]:
recipe_process_prompt = '''
You are a helpful assistant that can process the following recipe:

title = {0}
Instructions = {1}
Ingredients = {2}

The output must contains the following attributes:
- step_by_step_instructions: Instructions broken into steps.
- main_ingredients: Key ingredients. Only include ingredient name.
- less_important_ingredients: Less important ingredients. Only include ingredient name.
- cooking_methods: All required cooking methods (e.g., baking, frying)
- cooking_time: Estimated total cooking time
- required_tools: List of necessary tools
- dietary_labels: Dietary tags (e.g., vegan, gluten-free)
- nutrition_value: Please calculate the calories in for each recipe
- meet_daily_nutrition_need: Specify if the calcualted calories meet the daily nutrition need (~2,000 to 2,200 calories per day). Only output "true" or "false".

The output must be a string in JSON format that contains the above attributes. Do not need to specify the format type(i.e. json) at the beginning of the output string

'''

In [142]:
#Init the client
recipe_data_process_client = OpenAI(
    # This is the default and can be omitted
    api_key=api_key,
)
temp = 1
topp = 1

test_recipes = random.sample(list(recipe_dataset.items()), 10)
test_recipes = dict(test_recipes)

processed_recipe_list = get_processed_recipe_dataset(recipe_data_process_client,temp,topp,test_recipes)


In [143]:
with open("test_processed_recipes.json", "w") as file:
    json.dump(processed_recipe_list, file, indent=4)

print("processed_recipe_list has been saved")

processed_recipe_list has been saved


### Ingredient-Nutrient Mapping

Use https://produits-sante.canada.ca/api/documentation/cnf-documentation-en.html

Step1: Use food descriptio to get food_code

Step2: Use the food_code to get nutrient amount(s) (nutrient value and nutrient_name_id)

Step 3: Use nutrient_name_id to get nutrient_name


In [145]:
all_ingredients = set() #unique set

for eachRecipe in processed_recipe_list:
    main_ingredients = eachRecipe["processed_output"].get("main_ingredients", [])
    less_important_ingredients = eachRecipe["processed_output"].get("less_important_ingredients", [])

    all_ingredients.update(main_ingredients)
    all_ingredients.update(less_important_ingredients)


all_ingredients_list = sorted(all_ingredients)
print(all_ingredients_list)

['Ancho chile powder', 'Basic tomato sauce', 'Boneless rib-eyes', 'Butter', 'Canola oil', 'Carrot', 'Chicken stock', 'Chives', 'Cinzano Rosso sweet vermouth', 'Cream', 'Dijon mustard', 'Dried oregano', 'Dry mustard', 'Freshly ground pepper', 'Garlic', 'Ground coriander', 'Ground cumin', 'Honey', 'Horseradish', 'Ketchup', 'Kosher salt', 'Milk', 'Molasses', 'Old Fashioned Frosting', 'Olive oil', 'Osetra caviar', 'Pepper', 'Russet potatoes', 'Salt', 'Seasoned flour', 'Sour cream', 'Spanish onions', 'Speck', 'Sweet paprika', 'Thyme', 'Tomatoes', 'Venison', 'Waxy potatoes', 'Whole-grain mustard', 'all-purpose flour', 'bacon', 'baking powder', 'baking soda', 'black pepper', 'bok choy', 'butter', 'cake flour', 'cayenne pepper', 'celery seeds', 'cider vinegar', 'coarse salt', "confectioners' sugar", 'dry mustard', 'dry pectin', 'egg white', 'eggs', 'fresh flat-leafed parsley leaves', 'freshly grated nutmeg', 'freshly ground pepper', 'garlic', 'granulated sugar', 'ground anise', 'ground cinnamo